In [1]:
from cval import CVal

config_file = "../config.toml"
env_file = "../.env"

cval = CVal.init(
    config_file=config_file,
    env_file=env_file
)

/Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


[06/26/24 15:18:47] INFO     Ingestion engine initialized.                                          ]8;id=782062;file:///Users/sebastiansimon/GitHub/cval/src/ingestion.py\ingestion.py]8;;\:]8;id=391646;file:///Users/sebastiansimon/GitHub/cval/src/ingestion.py#42\42]8;;\

                    INFO     Retrieval engine initialized.                                          ]8;id=392544;file:///Users/sebastiansimon/GitHub/cval/src/retrieval.py\retrieval.py]8;;\:]8;id=955674;file:///Users/sebastiansimon/GitHub/cval/src/retrieval.py#29\29]8;;\

                    INFO     Ingestion engine initialized.                                          ]8;id=108272;file:///Users/sebastiansimon/GitHub/cval/src/ingestion.py\ingestion.py]8;;\:]8;id=572933;file:///Users/sebastiansimon/GitHub/cval/src/ingestion.py#42\42]8;;\

                    INFO     GPT (gpt-4o-2024-05-13) generator initialized.                         ]8;id=292553;file:///Users/sebastiansimon/GitHub/cval/src/generator.py\generator.py]8;;\:]8;id=640780;file:///Users/sebastiansimon/GitHub/cval/src/generator.py#47\47]8;;\

                    INFO     CVal initialized.                                                          ]8;id=762091;file:///Users/sebastiansimon/GitHub/cval/src/cval.py\cval.py]8;;\:]8;id=983400;file:///Users/sebastiansimon/GitHub/cval/src/cval.py#107\107]8;;\

In [2]:
import pandas as pd

eval_data_file = "../data/eval/dependencies_without_rules.csv"

df = pd.read_csv(eval_data_file, sep=";")

df_sample = df[:5]

df_sample

,link_str,project,option_name,option_value,option_file,option_type,option_technology,dependent_option_name,dependent_option_value,dependent_option_file,dependent_option_type,dependent_option_technology,pre_rating,rating,Notes
0,mall::::mall-admin/src/main/resources/applicat...,mall,spring.datasource.druid.stat-view-servlet.logi...,druid,mall-admin/src/main/resources/application-dev.yml,ConfigType.PASSWORD,spring,spring.datasource.druid.stat-view-servlet.logi...,druid,mall-search/src/main/resources/application-pro...,ConfigType.PASSWORD,spring,False,True,"borderline case, false if each service has it..."
1,mall::::mall-mbg/pom.xml::::project::::groupId...,mall,project.groupId,com.macro.mall,mall-mbg/pom.xml,ConfigType.NAME,maven,project.parent_mall.groupId,com.macro.mall,mall-mbg/pom.xml,ConfigType.NAME,maven,True,True,NaN
2,apollo::::apollo-biz/src/test/resources/applic...,apollo,spring.h2.console.enabled,true,apollo-biz/src/test/resources/application.prop...,ConfigType.BOOLEAN,spring,project.build.plugins.plugin_maven-jar-plugin....,true,pom.xml,ConfigType.IP_ADDRESS,maven,False,False,NaN
3,apollo::::apollo-assembly/pom.xml::::project::...,apollo,project.dependencies.dependency_apollo-portal....,apollo-portal,apollo-assembly/pom.xml,ConfigType.NAME,maven,COPY.src,apollo-portal,apollo-portal/src/main/docker/Dockerfile,ConfigType.PATH,docker,False,False,NaN
4,apollo::::apollo-assembly/pom.xml::::project::...,apollo,project.parent_apollo.relativePath,../pom.xml,apollo-assembly/pom.xml,ConfigType.PATH,maven,project.parent_apollo.relativePath,../pom.xml,apollo-portal/pom.xml,ConfigType.PATH,maven,NaN,True,NaN


In [6]:
from deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric, ContextualRelevancyMetric
from deepeval.test_case import LLMTestCase
from dotenv import dotenv_values
import os

config = dotenv_values(dotenv_path="../.env")
os.environ['OPENAI_API_KEY'] = config["OPENAI_KEY"]


from deepeval.models.base_model import DeepEvalBaseLLM
from llama_index.llms.ollama import Ollama

model = Ollama(model="llama3:70b", temperature=0.0, format="json")

class LlamaModel(DeepEvalBaseLLM):
    def __init__(
        self,
        model
    ):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        model = self.load_model()
        return model.complete(prompt).text

    async def a_generate(self, prompt: str) -> str:
        model = self.load_model()
        res = await model.acomplete(prompt)
        return res.text

    def get_model_name(self):
        return "Custom Llama3 model"


llama_model = LlamaModel(model=model)


answer_relevancy_metric = AnswerRelevancyMetric(
    threshold=0.7,
    model="gpt-4o",
    include_reason=True
)

faithfulness_metric = FaithfulnessMetric(
    threshold=0.7,
    model="gpt-4o",
    include_reason=True
)

context_relevancy_metric = ContextualRelevancyMetric(
    threshold=0.7,
    model="gpt-4o",
    include_reason=True
)

In [4]:
from data import Dependency
import mlflow
import datetime
import json

index_name = "so-posts"
exp_name = "dependencies_without_rules"
date_time = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")

mlflow.set_experiment(experiment_name=exp_name)

with mlflow.start_run(run_name=f"{exp_name}_{date_time}"):

    output = []

    for x in df_sample.to_dict("records"):
        dependency = Dependency(
            project=x["project"],
            option_name=x["option_name"],
            option_value=x["option_value"],
            option_type=x["option_type"].split(".")[-1],
            option_file=x["option_file"],
            option_technology=x["option_technology"],
            dependent_option_name=x["dependent_option_name"],
            dependent_option_value=x["dependent_option_value"],
            dependent_option_type=x["dependent_option_type"].split(".")[-1],
            dependent_option_file=x["dependent_option_file"],
            dependent_option_technology=x["dependent_option_technology"]
        )

        response = cval.query(
            dependency=dependency,
            index_name=index_name
        )

        output.append(response)

        print("Response: ", response)

    inputs = [x.input for x in output]
    results = [x.response for x in output]
    mlflow.log_table(data={"inputs": inputs, "outputs": results}, artifact_file="results.json")

    

[06/26/24 15:18:48] INFO     Select index: so-posts.                                                ]8;id=115650;file:///Users/sebastiansimon/GitHub/cval/src/retrieval.py\retrieval.py]8;;\:]8;id=446892;file:///Users/sebastiansimon/GitHub/cval/src/retrieval.py#47\47]8;;\

[06/26/24 15:18:51] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=699759;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=581031;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py#1026\1026]8;;\
                             "HTTP/1.1 200 OK"                                                                     

                    INFO     HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200  ]8;id=920508;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=595711;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py#1026\1026]8;;\
                             OK"                                                                                   

[06/26/24 15:18:52] INFO     Len of retrieved nodes of query 0: 5                                  ]8;id=634981;file:///Users/sebastiansimon/GitHub/cval/src/retrieval.py\retrieval.py]8;;\:]8;id=612723;file:///Users/sebastiansimon/GitHub/cval/src/retrieval.py#174\174]8;;\

[06/26/24 15:18:53] INFO     HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200  ]8;id=299420;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=947290;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py#1026\1026]8;;\
                             OK"                                                                                   

...

                    INFO     Len of retrieved nodes of query 2: 5                                  ]8;id=878556;file:///Users/sebastiansimon/GitHub/cval/src/retrieval.py\retrieval.py]8;;\:]8;id=242710;file:///Users/sebastiansimon/GitHub/cval/src/retrieval.py#174\174]8;;\

Batches: 100%|██████████| 1/1 [00:00<00:00, 21.06it/s]


[06/26/24 15:19:45] INFO     Len of retrieved nodes after reranking: 3                             ]8;id=16562;file:///Users/sebastiansimon/GitHub/cval/src/retrieval.py\retrieval.py]8;;\:]8;id=628080;file:///Users/sebastiansimon/GitHub/cval/src/retrieval.py#117\117]8;;\

[06/26/24 15:19:49] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=560451;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=807439;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py#1026\1026]8;;\
                             "HTTP/1.1 200 OK"                                                                     

Response:  { 
  "plan": "1. Identify the configuration options in question: project.parent_apollo.relativePath in apollo-assembly/pom.xml and project.parent_apollo.relativePath in apollo-portal/pom.xml. \n2. Verify the values of these configuration options: both are set to '../pom.xml'. \n3. Understand the purpose of the 'relativePath' configuration in Maven, which is to specify the path to the parent POM file relative to the current POM file. \n4. Determine if the functionality of one configuration option is directly dependent on the value of the other. \n5. Assess if the identical values are coincidental or if they must be the same for the build process to work correctly.",
  "rationale": "The 'relativePath' configuration in Maven specifies the path to the parent POM file. Both configuration options have the same value '../pom.xml', which indicates that they are pointing to the same parent POM file. This is a common practice in multi-module Maven projects where multiple modules share

In [7]:
faithfulness_scores = []
answer_relevancy_scores = []
context_relevancy_scores = []

false_positives = []
true_positives = []
false_negatives = []
true_negatives = []
accuracy = []

for response, baseline in zip(output, df_sample.to_dict("records")):


    test_case = LLMTestCase(
        input=response.input,
        actual_output=response.response,
        retrieval_context=[source_node.node.get_content() for source_node in response.source_nodes]
    )

    context_relevancy_metric.measure(test_case)
    faithfulness_metric.measure(test_case)
    answer_relevancy_metric.measure(test_case)

    context_relevancy_scores.append(context_relevancy_metric.score)
    faithfulness_scores.append(faithfulness_metric.score)
    answer_relevancy_scores.append(answer_relevancy_metric.score)       
        
print("Context Relevancy: ", context_relevancy_scores, sum(context_relevancy_scores)/len(context_relevancy_scores))
print("Answer Relevancy: ", answer_relevancy_scores, sum(answer_relevancy_scores)/len(answer_relevancy_scores))
print("Faithfulness: ", faithfulness_scores, sum(faithfulness_scores)/len(faithfulness_scores))

/Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Event loop is already running. Applying nest_asyncio patch to allow async execution...

/Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[06/26/24 15:39:23] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=618089;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=984585;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py#1773\1773]8;;\
                             "HTTP/1.1 200 OK"                                                                     

                    INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=800294;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=301079;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py#1773\1773]8;;\
                             "HTTP/1.1 200 OK"                                                                     

...

/Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[06/26/24 15:42:07] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=106599;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=796951;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py#1773\1773]8;;\
                             "HTTP/1.1 200 OK"                                                                     

/Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[06/26/24 15:42:08] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=329736;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=676621;file:///Users/sebastiansimon/GitHub/cval/env/lib/python3.12/site-packages/httpx/_client.py#1773\1773]8;;\
                             "HTTP/1.1 200 OK"                                                                     

Context Relevancy:  [0.0, 0.0, 0.0, 0.0, 0.0] 0.0
Answer Relevancy:  [0.6153846153846154, 0.8181818181818182, 0.3125, 0.7142857142857143, 0.9166666666666666] 0.6754037629037629
Faithfulness:  [1.0, 1.0, 1.0, 0.8, 1.0] 0.96


In [9]:
import re

true_positives = []
true_negatives = []
false_positives = []
false_negatives = []
accuracy = []

for response, baseline in zip(output, df_sample.to_dict("records")):

    response_dict = json.loads(response.response)

    if baseline["rating"] in ("True") and response_dict["isDependency"]:
        accuracy.append(1)
        true_positives.append(1)
    if baseline["rating"] in ("False") and not response_dict["isDependency"]:
        accuracy.append(1)
        true_negatives.append(1)
    if baseline["rating"] in ("True") and not response_dict["isDependency"]:
        accuracy.append(0)
        false_negatives.append(1)
    if baseline["rating"] in ("False") and response_dict["isDependency"]:
        accuracy.append(0)
        false_positives.append(1)


#precision = sum(true_positives)/(sum(true_positives)+sum(false_positives))
#recall = sum(true_positives)/(sum(true_positives)+sum(false_negatives))
#f1_score = 2 * (precision * recall) / (precision + recall)

print("Accuracy", sum(accuracy)/len(accuracy))
print("TP", sum(true_positives))
print("FP", sum(false_positives))
print("TN", sum(true_negatives))
print("FN", sum(false_negatives))
#print("Precision", precision)
#print("Recall", recall)
#print("F1 Score: ", f1_score)

Accuracy 0.8
TP 2
FP 0
TN 2
FN 1


In [ ]:
#inputs = [x.input for x in output]
results = [x.response for x in output]
rating = df_sample["rating"]

df_results = pd.DataFrame().from_dict(data={"outputs": results, "isDependency": rating})
df_results.to_csv(f"../data/results/test_{index_name}.csv", index=False)


In [ ]:
for x in output:
    print(x.input)
    print(x.response)
    break